---

In [3]:
import requests
import json
import math
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from qgridnext import show_grid
from datetime import datetime, timedelta
import sys  
import os
import pandas_ta as ta
import sklearn as sk
import datetime as dt
from skopt.space import Integer, Real, Categorical

# Ensure the directories are in the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Data_Management'))) #We have a double .. as we are in the Strategy subfolder
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Universe_Selection')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Signal_Generation')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Risk_Management')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Validation')))

# Import the modules
from data import Data
from calculations import Calculations, Metrics
from coarse import Coarse_1 as Coarse
from fine import Fine_1 as Fine
from entry_signal import Trend_Following, Mean_Reversion
from tail_risk import Stop_Loss, Take_Profit
from manage_trade import Manage_Trade
from position import Position
from testing import WFO
from costs import Costs

Skipping category 'layer-1', already processed.
Skipping category 'depin', already processed.
Skipping category 'proof-of-work-pow', already processed.
Skipping category 'proof-of-stake-pos', already processed.
Skipping category 'meme-token', already processed.
Skipping category 'dog-themed-coins', already processed.
Skipping category 'eth-2-0-staking', already processed.
Skipping category 'non-fungible-tokens-nft', already processed.
Skipping category 'governance', already processed.
Skipping category 'artificial-intelligence', already processed.
Skipping category 'infrastructure', already processed.
Skipping category 'layer-2', already processed.
Skipping category 'zero-knowledge-zk', already processed.
Skipping category 'storage', already processed.
Skipping category 'oracle', already processed.
Skipping category 'bitcoin-fork', already processed.
Skipping category 'restaking', already processed.
Skipping category 'rollup', already processed.
Skipping category 'metaverse', already p

In [4]:
# Specify the relative or absolute path to the CSV file
file_path = r"C:\Users\yassi\OneDrive\Documents\GitHub\Portfolio_1\Technical_Portfolio\Data_Management\all_data.csv"

# Read the CSV file
all_data = pd.read_csv(file_path, index_col=['date', 'coin'], parse_dates=['date'])
all_data

close      creturns         high  \
date                coin                                               
2017-08-17 05:00:00 BTCUSDT   4315.320000  1.506209e-03  4328.690000   
                    ETHUSDT    303.100006  4.940270e-03   303.279999   
2017-08-17 06:00:00 BTCUSDT   4324.350000  3.151810e-06  4345.450000   
2017-08-17 07:00:00 BTCUSDT   4349.990000  1.868776e-08  4349.990000   
                    ETHUSDT    307.959991  8.617874e-05   307.959991   
...                                   ...           ...          ...   
2024-12-27 23:00:00 SYSUSDT      0.112200  0.000000e+00     0.112400   
                    TRXUSDT      0.259400  0.000000e+00     0.259600   
                    VETUSDT      0.045760  0.000000e+00     0.045760   
                    WAXPUSDT     0.040910  0.000000e+00     0.040940   
                    XECUSDT      0.000035  0.000000e+00     0.000035   

                              log_return          low         open  \
date                coin                                             
2017-08-17 05:00:00 BTCUSDT    -6.498159  4291.370000  4308.830000   
                    ETHUSDT    -5.310335   300.000000   301.609985   
2017-08-17 06:00:00 BTCUSDT    -6.169374  4309.370000  4330.290000   
2017-08-17 07:00:00 BTCUSDT    -5.127863  4287.410000  4316.620000   
                    ETHUSDT    -4.048752   302.600006   302.679993   
...                                  ...          ...          ...   
2024-12-27 23:00:00 SYSUSDT    -5.632999     0.111100     0.111600   
                    TRXUSDT    -7.860292     0.258600     0.259400   
                    VETUSDT    -5.479996     0.045370     0.045580   
                    WAXPUSDT   -5.915972     0.040590     0.040770   
                    XECUSDT    -5.503282     0.000034     0.000034   

                                    price   returns        volume  \
date                coin                                            
2017-08-17 05:00:00 BTCUSDT   4315.320000  0.001506  2.323492e+01   
                    ETHUSDT    303.100006  0.004940  3.776725e+02   
2017-08-17 06:00:00 BTCUSDT   4324.350000  0.002093  7.229691e+00   
2017-08-17 07:00:00 BTCUSDT   4349.990000  0.005929  4.443249e+00   
                    ETHUSDT    307.959991  0.017444  7.547451e+02   
...                                   ...       ...           ...   
2024-12-27 23:00:00 SYSUSDT      0.112200  0.003578  7.290790e+05   
                    TRXUSDT      0.259400  0.000386  1.499596e+07   
                    VETUSDT      0.045760  0.004169  1.004372e+07   
                    WAXPUSDT     0.040910  0.002696  1.056349e+06   
                    XECUSDT      0.000035  0.004073  1.147830e+09   

                              volume_in_dollars  
date                coin                         
2017-08-17 05:00:00 BTCUSDT        1.002661e+05  
                    ETHUSDT        1.144725e+05  
2017-08-17 06:00:00 BTCUSDT        3.126371e+04  
2017-08-17 07:00:00 BTCUSDT        1.932809e+04  
                    ETHUSDT        2.324313e+05  
...                                         ...  
2024-12-27 23:00:00 SYSUSDT        8.180266e+04  
                    TRXUSDT        3.889953e+06  
                    VETUSDT        4.596008e+05  
                    WAXPUSDT       4.321524e+04  
                    XECUSDT        3.961160e+04  

[956964 rows x 10 columns]

In [5]:
start_time = dt.datetime(2024, 1, 1)
end_time = dt.datetime(2024, 3, 1)
timeframes = ['1w', '1d', '4h', '1h', '30m','15m', '5m', '1m']
index = 3 #It is better to choose the highest frequency for the backtest to be able to downsample
interval = timeframes[index]
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'ADAUSDT', 'XRPUSDT']
data = Data(symbols, interval, start_time, end_time).df
data

close  creturns        high  log_return  \
date                coin                                                    
2024-01-01 01:00:00 ADAUSDT      0.6023  1.007359      0.6029    0.007332   
                    BNBUSDT    315.3000  1.002863    316.0000    0.002859   
                    BTCUSDT  42613.5600  1.003257  42775.0000    0.003251   
                    ETHUSDT   2303.7200  1.003577   2306.6000    0.003570   
                    XRPUSDT      0.6185  1.003733      0.6190    0.003726   
...                                 ...       ...         ...         ...   
2024-02-29 23:00:00 ADAUSDT      0.6547  1.094999      0.6606    0.009515   
                    BNBUSDT    399.4000  1.270356    400.7000    0.008044   
                    BTCUSDT  61130.9800  1.439215  61536.9400   -0.001521   
                    ETHUSDT   3340.0900  1.455054   3358.9700    0.005247   
                    XRPUSDT      0.5867  0.952126      0.5898    0.004270   

                                    low        open       price   returns  \
date                coin                                                    
2024-01-01 01:00:00 ADAUSDT      0.5966      0.5979      0.6023  0.007359   
                    BNBUSDT    314.1000    314.3000    315.3000  0.002863   
                    BTCUSDT  42431.6500  42475.2300  42613.5600  0.003257   
                    ETHUSDT   2292.9000   2295.5200   2303.7200  0.003577   
                    XRPUSDT      0.6150      0.6161      0.6185  0.003733   
...                                 ...         ...         ...       ...   
2024-02-29 23:00:00 ADAUSDT      0.6482      0.6485      0.6547  0.009561   
                    BNBUSDT    396.0000    396.3000    399.4000  0.008077   
                    BTCUSDT  60998.5100  61224.0200  61130.9800 -0.001520   
                    ETHUSDT   3319.1800   3322.6000   3340.0900  0.005261   
                    XRPUSDT      0.5841      0.5843      0.5867  0.004279   

                                   volume  volume_in_dollars  
date                coin                                      
2024-01-01 01:00:00 ADAUSDT  2.607607e+06       1.570562e+06  
                    BNBUSDT  1.555812e+04       4.905474e+06  
                    BTCUSDT  1.196379e+03       5.098195e+07  
                    ETHUSDT  8.413426e+03       1.938218e+07  
                    XRPUSDT  9.029149e+06       5.584529e+06  
...                                   ...                ...  
2024-02-29 23:00:00 ADAUSDT  9.134539e+06       5.980383e+06  
                    BNBUSDT  1.887541e+04       7.538840e+06  
                    BTCUSDT  1.694180e+03       1.035669e+08  
                    ETHUSDT  1.463559e+04       4.888418e+07  
                    XRPUSDT  1.859032e+07       1.090694e+07  

[7195 rows x 10 columns]

---

### Fees:

An essential part of the backtesting is simulating the fees, as it could turn profitable strategies into losing ones. The main goal of backtesting is to simulate the live trading environment as much as possible do evaluate strategies performance, and since fees is part of any trading algo, then its presence makes it better to simulate live trading.


Thus:
1. We need a method that indicates when a buy or a sell order (a trades column)
2. another method that calculates the fees based on the maker and taker fees provided by the exchange
3. Apply the fees to strategy returns (important not to do it on the cumulative strategy returns as this is the column used to calculate the objective)

Idea:
1. Calculates trades without abs value -> make sure to account for partials\
(The difference betwenn this one and the one in the calculations accounts for partials whereas the other one is used to made the sessions column)
2. If entry (trade is positive) then strategy return - trade * maker fees\
(and vice versa for when the trade is negative, but make sure to take the abs value of the trade)

In [6]:
# Generate random returns
np.random.seed(42)  # For reproducibility
data['strategy'] = np.random.randn(len(data))

In [7]:
data['position'] = np.random.randint(0, 2, size=len(data))

In [13]:
data['strategy'].unstack()

coin,ADAUSDT,BNBUSDT,BTCUSDT,ETHUSDT,XRPUSDT
date,,,,,
2024-01-01 01:00:00,0.496714,-0.138264,0.647689,1.523030,-0.234153
2024-01-01 02:00:00,-0.234137,1.579213,0.767435,-0.469474,0.542560
2024-01-01 03:00:00,-0.463418,-0.465730,0.241962,-1.913280,-1.724918
2024-01-01 04:00:00,-0.562288,-1.012831,0.314247,-0.908024,-1.412304
2024-01-01 05:00:00,1.465649,-0.225776,0.067528,-1.424748,-0.544383
...,...,...,...,...,...
2024-02-29 19:00:00,-0.483659,-0.246724,-0.168420,0.603646,1.374611
2024-02-29 20:00:00,0.359681,1.187078,-0.964790,-0.414631,-0.724303
2024-02-29 21:00:00,0.314445,-0.358958,0.729773,-0.931002,0.043031


In [11]:
#Calculate the trade_costs
maker_fees = 0.25
taker_fees = 0.40
_df = data.copy().unstack()


for coin in _df['close'].columns:
    _df['trades_with_partials', coin] = _df['position', coin].diff().fillna(0)
    _df['trade_costs', coin] = np.where(
        _df['trades_with_partials', coin] > 0,
        _df['trades_with_partials', coin] * (maker_fees / 100),
        _df['trades_with_partials', coin] * (taker_fees / 100) * (-1)
    )
    _df['strategy', coin] = _df['strategy', coin] - _df['trade_costs', coin]




In [12]:
_df['strategy']

coin,ADAUSDT,BNBUSDT,BTCUSDT,ETHUSDT,XRPUSDT
date,,,,,
2024-01-01 01:00:00,0.496714,-0.138264,0.647689,1.523030,-0.234153
2024-01-01 02:00:00,-0.236637,1.575213,0.764935,-0.471974,0.538560
2024-01-01 03:00:00,-0.463418,-0.465730,0.237962,-1.913280,-1.724918
2024-01-01 04:00:00,-0.562288,-1.012831,0.311747,-0.908024,-1.414804
2024-01-01 05:00:00,1.465649,-0.228276,0.063528,-1.428748,-0.544383
...,...,...,...,...,...
2024-02-29 19:00:00,-0.486159,-0.246724,-0.170920,0.603646,1.374611
2024-02-29 20:00:00,0.355681,1.187078,-0.968790,-0.414631,-0.728303
2024-02-29 21:00:00,0.311945,-0.358958,0.729773,-0.931002,0.043031


Putting All together:

In [18]:
def apply_fees(df, maker = 0.25, taker = 0.40):
    _df = df.copy().unstack()


    for coin in _df['close'].columns:
        _df['trades_with_partials', coin] = _df['position', coin].diff().fillna(0)
        _df['trade_costs', coin] = np.where(
            _df['trades_with_partials', coin] > 0,
            _df['trades_with_partials', coin] * (maker / 100),
            _df['trades_with_partials', coin] * (taker / 100) * (-1)
        )
        _df['strategy', coin] = _df['strategy', coin] - _df['trade_costs', coin]
        
    return _df.stack(future_stack=True)

In [19]:
apply_fees(data)

close  creturns        high  log_return  \
date                coin                                                    
2024-01-01 01:00:00 ADAUSDT      0.6023  1.007359      0.6029    0.007332   
                    BNBUSDT    315.3000  1.002863    316.0000    0.002859   
                    BTCUSDT  42613.5600  1.003257  42775.0000    0.003251   
                    ETHUSDT   2303.7200  1.003577   2306.6000    0.003570   
                    XRPUSDT      0.6185  1.003733      0.6190    0.003726   
...                                 ...       ...         ...         ...   
2024-02-29 23:00:00 ADAUSDT      0.6547  1.094999      0.6606    0.009515   
                    BNBUSDT    399.4000  1.270356    400.7000    0.008044   
                    BTCUSDT  61130.9800  1.439215  61536.9400   -0.001521   
                    ETHUSDT   3340.0900  1.455054   3358.9700    0.005247   
                    XRPUSDT      0.5867  0.952126      0.5898    0.004270   

                                    low        open       price   returns  \
date                coin                                                    
2024-01-01 01:00:00 ADAUSDT      0.5966      0.5979      0.6023  0.007359   
                    BNBUSDT    314.1000    314.3000    315.3000  0.002863   
                    BTCUSDT  42431.6500  42475.2300  42613.5600  0.003257   
                    ETHUSDT   2292.9000   2295.5200   2303.7200  0.003577   
                    XRPUSDT      0.6150      0.6161      0.6185  0.003733   
...                                 ...         ...         ...       ...   
2024-02-29 23:00:00 ADAUSDT      0.6482      0.6485      0.6547  0.009561   
                    BNBUSDT    396.0000    396.3000    399.4000  0.008077   
                    BTCUSDT  60998.5100  61224.0200  61130.9800 -0.001520   
                    ETHUSDT   3319.1800   3322.6000   3340.0900  0.005261   
                    XRPUSDT      0.5841      0.5843      0.5867  0.004279   

                                   volume  volume_in_dollars  strategy  \
date                coin                                                 
2024-01-01 01:00:00 ADAUSDT  2.607607e+06       1.570562e+06  0.496714   
                    BNBUSDT  1.555812e+04       4.905474e+06 -0.138264   
                    BTCUSDT  1.196379e+03       5.098195e+07  0.647689   
                    ETHUSDT  8.413426e+03       1.938218e+07  1.523030   
                    XRPUSDT  9.029149e+06       5.584529e+06 -0.234153   
...                                   ...                ...       ...   
2024-02-29 23:00:00 ADAUSDT  9.134539e+06       5.980383e+06  0.155594   
                    BNBUSDT  1.887541e+04       7.538840e+06  0.030943   
                    BTCUSDT  1.694180e+03       1.035669e+08 -0.085894   
                    ETHUSDT  1.463559e+04       4.888418e+07  0.991191   
                    XRPUSDT  1.859032e+07       1.090694e+07  0.743445   

                             position  trades_with_partials  trade_costs  
date                coin                                                  
2024-01-01 01:00:00 ADAUSDT         0                   0.0      -0.0000  
                    BNBUSDT         1                   0.0      -0.0000  
                    BTCUSDT         0                   0.0      -0.0000  
                    ETHUSDT         0                   0.0      -0.0000  
                    XRPUSDT         1                   0.0      -0.0000  
...                               ...                   ...          ...  
2024-02-29 23:00:00 ADAUSDT         1                   1.0       0.0025  
                    BNBUSDT         1                   1.0       0.0025  
                    BTCUSDT         0                  -1.0       0.0040  
                    ETHUSDT         0                   0.0      -0.0000  
                    XRPUSDT         0                  -1.0       0.0040  

[7195 rows x 14 columns]

---